## Probability that team A wins over team B assuming xGoals is Poisson distributed

In [77]:
import numpy as np
import pandas as pd
from scipy.stats import poisson

In [78]:
def get_poisson_win_rate(xgA, xgB):
    # Assume that  NHL teams score between 0 and 25 goals per game
    num_goals = [n for n in range(25)]

    # Get probabilities of scoring 0-20 from poisson distribution with mean xGoals
    pois_probsA = poisson.pmf(k = num_goals, mu = xgA)
    pois_probsB = poisson.pmf(k = num_goals, mu = xgB)

    # Set up df to store probabilities
    poisA = pd.DataFrame(zip(num_goals, pois_probsA), columns = ['goalsA', 'probA'])
    poisA['temp_key'] = 1

    poisB = pd.DataFrame(zip(num_goals, pois_probsB), columns = ['goalsB', 'probB'])
    poisB['temp_key'] = 1

    # Join on temporary key to get cross join
    all_outcomes = pd.merge(poisA, poisB, how = 'inner', on = 'temp_key').drop(columns = 'temp_key')

    # Calculate probability of each possible outcome
    all_outcomes['outcome_prob'] = all_outcomes['probA'] * all_outcomes['probB']

    # Get probability that team A wins. Pr(A win) = SUM(A > B) + 0.5 * SUM(A = B)
    prob_A_win = all_outcomes.loc[all_outcomes['goalsA'] > all_outcomes['goalsB'], 'outcome_prob'].sum() + 0.5 * all_outcomes.loc[all_outcomes['goalsA'] == all_outcomes['goalsB'], 'outcome_prob'].sum()

    return prob_A_win

### Test function

In [79]:
# Test using 2 values
get_poisson_win_rate(4.53, 3.89)

0.5860340625506661

In [80]:
# Test using df of values
test = pd.DataFrame({'teamA':['DET', 'WPG', 'VGK'],
                     'xgA':[3.17, 5.45, 2.58],
                     'teamB':['PIT', 'NYR', 'COL'],
                     'xgB':[4.32, 3.33, 2.81]
                     })

test['prob_teamA'] = test.apply(lambda row: get_poisson_win_rate(row['xgA'], row['xgB']), axis = 1)

test['prob_teamB'] = 1 - test['prob_teamA']

display(test)

,teamA,xgA,teamB,xgB,prob_teamA,prob_teamB
0,DET,3.17,PIT,4.32,0.339379,0.660621
1,WPG,5.45,NYR,3.33,0.761170,0.238830
2,VGK,2.58,COL,2.81,0.461483,0.538517
